In [42]:
import pandas as pd
import numpy as np

#Importing libraries  for visualisation of data
import matplotlib

#Importing sklearn libraries for modelling and evaluation

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

from sklearn.impute import SimpleImputer


from sklearn.preprocessing import MinMaxScaler ## Importing the MinMax Scaler

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_curve, roc_auc_score, auc, mean_squared_log_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/MyDrive/Trees")

In [90]:
ppf = pd.read_csv('Scores_main.csv')

X = ppf.drop(["Originality"],axis=1)
y = ppf["Originality"]

train_x,test_x,train_y,test_y = train_test_split(X,y, test_size=0.15, random_state = 50)

print('training data    ',train_x.shape,train_y.shape)
print('test data        ',test_x.shape,test_y.shape)

training data     (566, 5) (566,)
test data         (101, 5) (101,)


RANDOM FOREST

Base model with continous predictions

In [91]:
#Importing random forest classifier 
from sklearn.ensemble import RandomForestRegressor

In [92]:
rfr = RandomForestRegressor(random_state = 50)

In [93]:
#train the model
rfr.fit(train_x, train_y)

RandomForestRegressor(random_state=50)

In [94]:
#score on training data
rfr.score(train_x, train_y)

0.9481844988254209

In [95]:
#score on test data
rfr.score(test_x, test_y)

0.5265627660244451

In [96]:
y_pred_test = rfr.predict(test_x)

In [100]:
errors = mse(y_pred_test, test_y)
errors

0.07553404192169426

In [17]:
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rfr.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

In [101]:
import pandas as pd
rfr_feature_importances = pd.DataFrame(rfr.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [113]:
rfr_feature_importances

,importance
Grammarly,0.368423
Perplexity_Score,0.351999
Burstiness_Score,0.102022
OpenAI,0.096760
DetectGPT,0.080796


Regressor with GridSearchCV

In [103]:
from sklearn.model_selection import GridSearchCV

In [104]:
param_gridrfct1gsc = { 
    'n_estimators': [150, 200, 250],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,8,12,16,20, None],
    'criterion' :['squared_error', 'friedman_mse']
}


In [105]:
CV_rfc = GridSearchCV(estimator=rfr, param_grid=param_gridrfct1gsc, cv= 5,return_train_score=True)
CV_rfc.fit(train_x, train_y)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors an

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=50),
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [4, 8, 12, 16, 20, None],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [150, 200, 250]},
             return_train_score=True)

In [106]:
print(CV_rfc.best_params_)

{'criterion': 'squared_error', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 250}


In [107]:
#creating a random forest instance
rft1 = RandomForestRegressor(random_state=50, bootstrap=True, max_depth=8, max_features= 'sqrt', n_estimators= 250, criterion='squared_error')

In [108]:
#train the model
rft1.fit(train_x,train_y)

RandomForestRegressor(max_depth=8, max_features='sqrt', n_estimators=250,
                      random_state=50)

In [109]:
#score on training data
rft1.score(train_x, train_y)

0.8629002263161816

In [110]:
#score on test data
rft1.score(test_x, test_y)

0.538815019601713

In [111]:
y_pred_testt1 = rft1.predict(test_x)